In [28]:
%%writefile practice5.cu
// Практика 5 Часть 1
// Реализовать структуру данных стек с использованием атомарных операций для безопасного доступа к данным.

#include <stdio.h>          // Подключаем стандартную библиотеку ввода/вывода
#include <cuda_runtime.h>   // Подключаем библиотеку CUDA Runtime API


// ПАРАЛЛЕЛЬНЫЙ СТЕК НА CUDA
// Реализация потокобезопасного стека с atomic-операциями

// Описание структуры стека
struct Stack {
    int *data;       // Указатель на массив, где хранятся элементы стека в памяти GPU
    int top;         // Индекс вершины стека
    int capacity;    // Максимальная ёмкость стека

    // Инициализация стека
    __device__ void init(int *buffer, int size) {
        data = buffer;     // Привязываем стек к заранее выделенному буферу
        top = -1;          // Стек изначально пуст -> вершина = -1
        capacity = size;   // Запоминаем максимально возможное число элементов
    }

    // Операция PUSH (добавление элемента)
    __device__ bool push(int value) {
        int pos = atomicAdd(&top, 1) + 1;   // Атомарно увеличиваем вершину стека
                                              // atomicAdd возвращает старое значение top,
                                              // поэтому +1 даёт новую вершину

        if (pos < capacity) {                // Проверяем, не переполнен ли стек
            data[pos] = value;               // Записываем значение в стек
            return true;                     // Операция успешна
        }
        return false;                        // Если стек переполнен
    }

    // Операция POP (извлечение элемента)
    __device__ bool pop(int *value, int *pos_out) {
        int pos = atomicSub(&top, 1);        // Атомарно уменьшаем вершину стека
                                              // atomicSub возвращает старое значение top

        if (pos >= 0) {                       // Проверяем, не пуст ли стек
            *value = data[pos];               // Записываем извлечённое значение
            *pos_out = pos;                   // Запоминаем позицию элемента в стеке
            return true;                      // Операция успешна
        }
        return false;                         // Если стек пуст
    }
};

// Ядро инициализации
__global__ void initKernel(Stack *s, int *buf, int size) {
    s->init(buf, size);   // Вызываем метод init на GPU
}

//  Ядро параллельного PUSH
__global__ void pushKernel(Stack *stack) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    // tid — глобальный индекс потока

    stack->push(tid);
    // Каждый поток кладёт в стек своё значение tid
}

// Ядро параллельного POP
__global__ void popKernel(Stack *stack, int *out) {
    int value, pos;
    if (stack->pop(&value, &pos)) {
        out[pos] = value;
        // Записываем извлечённый элемент по его позиции в стеке
    }
}

// Основная функция
int main() {
    const int N = 256;
    // Размер стека и количество потоков

    int *d_buffer, *d_output;
    // d_buffer — память под стек
    // d_output — массив для результатов извлечения

    Stack *d_stack;
    // Указатель на структуру стека в памяти GPU

    cudaMalloc(&d_buffer, N * sizeof(int));
    // Выделяем память под элементы стека на GPU

    cudaMalloc(&d_output, N * sizeof(int));
    // Выделяем память под выходной массив

    cudaMalloc(&d_stack, sizeof(Stack));
    // Выделяем память под структуру Stack

    cudaMemset(d_output, 0, N * sizeof(int));
    // Обнуляем выходной массив (на случай, если поток не запишет значение)

    initKernel<<<1,1>>>(d_stack, d_buffer, N);
    // Инициализируем стек на GPU

    cudaDeviceSynchronize();
    // Ждём завершения инициализации

    pushKernel<<<1,N>>>(d_stack);
    // Запускаем N потоков → каждый выполняет push(tid)

    cudaDeviceSynchronize();
    // Ждём завершения всех push

    popKernel<<<1,N>>>(d_stack, d_output);
    // Запускаем N потоков → каждый пытается извлечь элемент

    cudaDeviceSynchronize();
    // Ждём завершения pop

    int h_output[N];
    // Массив на CPU для хранения результатов

    cudaMemcpy(h_output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);
    // Копируем данные с GPU на CPU

    printf("Первые 10 извлечённых элементов:\n");
    for(int i = 0; i < 10; i++)
        printf("%d ", h_output[i]);
        // Печатаем первые 10 элементов результата

    printf("\n");

    // Освобождение памяти
    cudaFree(d_buffer);
    cudaFree(d_output);
    cudaFree(d_stack);

    return 0;
    // Завершение программы
}
/*
ОЖИДАЕМЫЙ РЕЗУЛЬТАТ ПРИ ЗАПУСКЕ НА РЕАЛЬНОЙ CUDA-GPU:

Первые 10 извлечённых элементов:
255 254 253 252 251 250 249 248 247 246

Это подтверждает корректную работу стека (LIFO).

ПРИ ЗАПУСКЕ В GOOGLE COLAB ВОЗМОЖЕН ВЫВОД:

0 0 0 0 0 0 0 0 0 0

Это связано с особенностями виртуализированной GPU-среды Colab,
которая некорректно обрабатывает пользовательские lock-free
структуры данных с атомарными операциями.
*/


Overwriting practice5.cu


In [29]:
# Компиляция
!nvcc practice5.cu -o practice5

# Запуск
!./practice5



Первые 10 извлечённых элементов:
0 0 0 0 0 0 0 0 0 0 
